In [1]:
!pip install fuzzywuzzy
!pip install tqdm
!pip install recordlinkage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.5/926.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import regex as re
from tqdm import tqdm
import pandas as pd
import recordlinkage

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
df0 = pd.read_csv('/content/drive/MyDrive/2022_2/Data_Integration/Train_1_10/data_final.csv')

In [4]:
df0.head()

,title,price,address,area,date,description,link_image,url_page,direct,floor,name_contact,phone_contact
0,Căn Hộ 2 PN 55m2 Phan Huy Ích – sát KCN Tân B...,880.0,"Phường 15, Quận Tân Bình, TP Hồ Chí Minh",55.0m2,NaN,NaN,['https://img.homedy.com/store/images/2023/06/...,https://homedy.com/ban-can-ho-quan-tan-binh-tp...,NaN,NaN,Quý,0901.321.245
1,Căn Hộ Q Tân Bình 860tr/sổ riêng-Trọn đời. Tặ...,860.0,"Phường 13, Quận Tân Bình, TP Hồ Chí Minh",60.0m2,NaN,NaN,['https://img.homedy.com/store/images/2023/06/...,https://homedy.com/ban-can-ho-quan-tan-binh-tp...,NaN,NaN,Trần Mai Phương,0931.601.365
2,Căn Hộ 1PN Full Nội Thất 38m2 CẠNH SÔNG SÀI G...,11.5,"Phường Bình An, Quận 2, TP Hồ Chí Minh",38.0m2,NaN,NaN,['https://img.homedy.com/store/images/2023/06/...,https://homedy.com/ban-can-ho-quan-2-tp-ho-chi...,NaN,NaN,Nguyễn Thị Tuyết Hương,0906.022.702
3,Bán căn hộ Vũng Tàu Center Thành phố Vũng Tàu...,2650.0,"Phường Thắng Nhì, Thành phố Vũng Tàu, Bà Rịa ...",76.0m2,NaN,- Ngân Hàng tài trợ 70%,['https://img.homedy.com/store/images/2023/03/...,https://homedy.com/ban-can-ho-thanh-pho-vung-t...,NaN,NaN,Son,0937.002.378
4,Bán căn hộ Thành phố Đà Lạt - Lâm Đồng giá 2....,2900.0,"Phường 10, Thành phố Đà Lạt, Lâm Đồng",56.0m2,NaN,NaN,['https://img.homedy.com/store/images/2023/06/...,https://homedy.com/ban-can-ho-thanh-pho-da-lat...,NaN,NaN,Liêm ngô,0937.500.334


In [5]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64562 entries, 0 to 64561
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          64519 non-null  object 
 1   price          64521 non-null  float64
 2   address        64521 non-null  object 
 3   area           62335 non-null  object 
 4   date           59973 non-null  object 
 5   description    60833 non-null  object 
 6   link_image     64562 non-null  object 
 7   url_page       64562 non-null  object 
 8   direct         4211 non-null   object 
 9   floor          6918 non-null   float64
 10  name_contact   29542 non-null  object 
 11  phone_contact  64560 non-null  object 
dtypes: float64(2), object(10)
memory usage: 5.9+ MB


In [6]:
df = ((df0.dropna(subset=['title'])).dropna(subset=['area']).dropna(subset=['description']))

In [7]:
df = df0

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64562 entries, 0 to 64561
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          64519 non-null  object 
 1   price          64521 non-null  float64
 2   address        64521 non-null  object 
 3   area           62335 non-null  object 
 4   date           59973 non-null  object 
 5   description    60833 non-null  object 
 6   link_image     64562 non-null  object 
 7   url_page       64562 non-null  object 
 8   direct         4211 non-null   object 
 9   floor          6918 non-null   float64
 10  name_contact   29542 non-null  object 
 11  phone_contact  64560 non-null  object 
dtypes: float64(2), object(10)
memory usage: 5.9+ MB


In [9]:
def cut_city(addr):
    if(type(addr) == float):
            return ''
    return addr.split(',')[-1]

def cut_district(addr):
    if(type(addr) == float):
            return ''
    if(len(addr.split(',')) < 2):
        return ''
    return addr.split(',')[-2]

df['city'] = df['address'].apply(cut_city)
data_city = df['city'].unique()
print(data_city)
print(len(data_city))

df['district'] = df['address'].apply(cut_district)
data_district = df['district'].unique()
print(data_district)
print(len(data_district))

[' TP Hồ Chí Minh' ' Bà Rịa Vũng Tàu' ' Lâm Đồng' ' Đà Nẵng' ' Hà Nội'
 ' Bình Dương' ' Bình Định' ' Đồng Nai' 'Hà Nội' 'Bình Định'
 'TP Hồ Chí Minh' ' Hải Phòng' ' Khánh Hòa' '' ' Quảng Ninh' 'Khánh Hòa'
 ' Kiên Giang' ' Nam Định' ' Hưng Yên' 'Đà Nẵng' ' Thanh Hóa' ' Bắc Ninh'
 ' Bình Thuận' 'Đồng Nai' ' Nghệ An' ' Bắc Giang' ' Quảng Bình' ' Phú Thọ'
 ' Long An' 'Bình Dương' ' Tây Ninh' 'Hải Phòng' ' Hồ Chí Minh' ' Cần Thơ'
 ' Huế' ' Bến Tre' ' Ninh Thuận' ' Vĩnh Phúc' ' Thái Nguyên' ' Hải Dương'
 ' Đắk Lắk' ' Bắc Cạn' ' Kon Tum' ' Quảng Nam' ' Hòa Bình' ' Bạc Liêu'
 ' Hậu Giang' ' Yên Bái' ' Thái Bình' ' Lào Cai' ' An Giang'
 ' Bà Rịa - Vũng Tàu' ' TP. Hồ Chí Minh' ' TP. Hải Phòng' ' TP. Đà Nẵng'
 ' Hà Tĩnh' ' TPHCM' ' TP.HCM' ' Cà Mau' ' Thừa Thiên Huế' ' Vĩnh Long'
 ' Tiền Giang' ' Bình Phước' ' Quảng Ngãi' ' Ha Noi' ' TP HCM' ' ']
67
['  Quận Tân Bình' '  Quận 2' '  Thành phố Vũng Tàu' '  Thành phố Đà Lạt'
 '  Quận Ngũ Hành Sơn' '  Quận Ba Đình' '  Thành phố Thuận An'
 '  Huyện Bì

In [10]:
df["city"] = df["city"].apply(lambda x: re.sub(r"\s+", " ", x.strip()))
df["district"] = df["district"].apply(lambda x: re.sub(r"\s+", " ", x.strip()))

In [11]:
data_city = df['city'].unique()
print(data_city)
print(len(data_city))

data_district = df['district'].unique()
print(data_district)
print(len(data_district))

['TP Hồ Chí Minh' 'Bà Rịa Vũng Tàu' 'Lâm Đồng' 'Đà Nẵng' 'Hà Nội'
 'Bình Dương' 'Bình Định' 'Đồng Nai' 'Hải Phòng' 'Khánh Hòa' ''
 'Quảng Ninh' 'Kiên Giang' 'Nam Định' 'Hưng Yên' 'Thanh Hóa' 'Bắc Ninh'
 'Bình Thuận' 'Nghệ An' 'Bắc Giang' 'Quảng Bình' 'Phú Thọ' 'Long An'
 'Tây Ninh' 'Hồ Chí Minh' 'Cần Thơ' 'Huế' 'Bến Tre' 'Ninh Thuận'
 'Vĩnh Phúc' 'Thái Nguyên' 'Hải Dương' 'Đắk Lắk' 'Bắc Cạn' 'Kon Tum'
 'Quảng Nam' 'Hòa Bình' 'Bạc Liêu' 'Hậu Giang' 'Yên Bái' 'Thái Bình'
 'Lào Cai' 'An Giang' 'Bà Rịa - Vũng Tàu' 'TP. Hồ Chí Minh'
 'TP. Hải Phòng' 'TP. Đà Nẵng' 'Hà Tĩnh' 'TPHCM' 'TP.HCM' 'Cà Mau'
 'Thừa Thiên Huế' 'Vĩnh Long' 'Tiền Giang' 'Bình Phước' 'Quảng Ngãi'
 'Ha Noi' 'TP HCM']
58
['Quận Tân Bình' 'Quận 2' 'Thành phố Vũng Tàu' 'Thành phố Đà Lạt'
 'Quận Ngũ Hành Sơn' 'Quận Ba Đình' 'Thành phố Thuận An'
 'Huyện Bình Chánh' 'Quận Nam Từ Liêm' 'Quận Tân Phú' 'Quận Long Biên'
 'Quận 6' 'Huyện Gia Lâm' 'Huyện Đông Anh' 'Quận Gò Vấp'
 'Thành phố Quy Nhơn' 'Quận Hai Bà Trưng' 'Thành phố Biê

In [12]:
def convert_city(city):
    dict_city ={'TP Hồ Chí Minh':'Hồ Chí Minh', 'TPHCM':'Hồ Chí Minh', 'TP.HCM':'Hồ Chí Minh',
                'TP HCM':'Hồ Chí Minh', 'TP. Hồ Chí Minh':'Hồ Chí Minh', 'Ha Noi':'Hà Nội',
                'Huế':'Thừa Thiên Huế', 'Bà Rịa Vũng Tàu':'Bà Rịa - Vũng Tàu',
                'TP. Hải Phòng':'Hải Phòng', 'TP. Đà Nẵng':'Đà Nẵng'}
    if(city in dict_city):
        return dict_city[city]
    else:
        return city
df['city'] = df['city'].apply(convert_city)

In [13]:
data_city = df['city'].unique()
print(data_city)
print(len(data_city))

data_district = df['district'].unique()
print(data_district)
print(len(data_district))

['Hồ Chí Minh' 'Bà Rịa - Vũng Tàu' 'Lâm Đồng' 'Đà Nẵng' 'Hà Nội'
 'Bình Dương' 'Bình Định' 'Đồng Nai' 'Hải Phòng' 'Khánh Hòa' ''
 'Quảng Ninh' 'Kiên Giang' 'Nam Định' 'Hưng Yên' 'Thanh Hóa' 'Bắc Ninh'
 'Bình Thuận' 'Nghệ An' 'Bắc Giang' 'Quảng Bình' 'Phú Thọ' 'Long An'
 'Tây Ninh' 'Cần Thơ' 'Thừa Thiên Huế' 'Bến Tre' 'Ninh Thuận' 'Vĩnh Phúc'
 'Thái Nguyên' 'Hải Dương' 'Đắk Lắk' 'Bắc Cạn' 'Kon Tum' 'Quảng Nam'
 'Hòa Bình' 'Bạc Liêu' 'Hậu Giang' 'Yên Bái' 'Thái Bình' 'Lào Cai'
 'An Giang' 'Hà Tĩnh' 'Cà Mau' 'Vĩnh Long' 'Tiền Giang' 'Bình Phước'
 'Quảng Ngãi']
48
['Quận Tân Bình' 'Quận 2' 'Thành phố Vũng Tàu' 'Thành phố Đà Lạt'
 'Quận Ngũ Hành Sơn' 'Quận Ba Đình' 'Thành phố Thuận An'
 'Huyện Bình Chánh' 'Quận Nam Từ Liêm' 'Quận Tân Phú' 'Quận Long Biên'
 'Quận 6' 'Huyện Gia Lâm' 'Huyện Đông Anh' 'Quận Gò Vấp'
 'Thành phố Quy Nhơn' 'Quận Hai Bà Trưng' 'Thành phố Biên Hòa'
 'Quận Hoàng Mai' 'Quận Đống Đa' '' 'Quận 12' 'Quận Bình Thạnh'
 'Quận Thanh Xuân' 'Quận Hà Đông' 'Quận 7' 'Quận Sơn Tr

In [14]:
df = df.drop(df[df['city'] == ''].index)

In [15]:
data_city = df['city'].unique()
print(data_city)
print(len(data_city))

data_district = df['district'].unique()
print(data_district)
print(len(data_district))

['Hồ Chí Minh' 'Bà Rịa - Vũng Tàu' 'Lâm Đồng' 'Đà Nẵng' 'Hà Nội'
 'Bình Dương' 'Bình Định' 'Đồng Nai' 'Hải Phòng' 'Khánh Hòa' 'Quảng Ninh'
 'Kiên Giang' 'Nam Định' 'Hưng Yên' 'Thanh Hóa' 'Bắc Ninh' 'Bình Thuận'
 'Nghệ An' 'Bắc Giang' 'Quảng Bình' 'Phú Thọ' 'Long An' 'Tây Ninh'
 'Cần Thơ' 'Thừa Thiên Huế' 'Bến Tre' 'Ninh Thuận' 'Vĩnh Phúc'
 'Thái Nguyên' 'Hải Dương' 'Đắk Lắk' 'Bắc Cạn' 'Kon Tum' 'Quảng Nam'
 'Hòa Bình' 'Bạc Liêu' 'Hậu Giang' 'Yên Bái' 'Thái Bình' 'Lào Cai'
 'An Giang' 'Hà Tĩnh' 'Cà Mau' 'Vĩnh Long' 'Tiền Giang' 'Bình Phước'
 'Quảng Ngãi']
47
['Quận Tân Bình' 'Quận 2' 'Thành phố Vũng Tàu' 'Thành phố Đà Lạt'
 'Quận Ngũ Hành Sơn' 'Quận Ba Đình' 'Thành phố Thuận An'
 'Huyện Bình Chánh' 'Quận Nam Từ Liêm' 'Quận Tân Phú' 'Quận Long Biên'
 'Quận 6' 'Huyện Gia Lâm' 'Huyện Đông Anh' 'Quận Gò Vấp'
 'Thành phố Quy Nhơn' 'Quận Hai Bà Trưng' 'Thành phố Biên Hòa'
 'Quận Hoàng Mai' 'Quận Đống Đa' '' 'Quận 12' 'Quận Bình Thạnh'
 'Quận Thanh Xuân' 'Quận Hà Đông' 'Quận 7' 'Quận Sơn Trà'


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64508 entries, 0 to 64561
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          64506 non-null  object 
 1   price          64508 non-null  float64
 2   address        64508 non-null  object 
 3   area           62322 non-null  object 
 4   date           59973 non-null  object 
 5   description    60830 non-null  object 
 6   link_image     64508 non-null  object 
 7   url_page       64508 non-null  object 
 8   direct         4211 non-null   object 
 9   floor          6918 non-null   float64
 10  name_contact   29488 non-null  object 
 11  phone_contact  64506 non-null  object 
 12  city           64508 non-null  object 
 13  district       64508 non-null  object 
dtypes: float64(2), object(12)
memory usage: 7.4+ MB


In [17]:
count_value = df['city'].value_counts()
print(count_value)

Hồ Chí Minh          42818
Hà Nội               13833
Bình Dương            2610
Đà Nẵng               1303
Bà Rịa - Vũng Tàu      748
Đồng Nai               468
Khánh Hòa              413
Bình Định              393
Thanh Hóa              373
Bắc Ninh               344
Quảng Ninh             254
Hải Phòng              224
Bình Thuận             164
Bắc Giang              107
Thừa Thiên Huế          62
Vĩnh Phúc               58
Hưng Yên                57
Bến Tre                 40
Lâm Đồng                35
Nghệ An                 33
Cần Thơ                 30
Tây Ninh                23
Long An                 14
Phú Thọ                 14
Thái Nguyên             12
Hậu Giang               12
Kiên Giang              11
Ninh Thuận               7
Quảng Nam                5
Vĩnh Long                4
An Giang                 4
Hòa Bình                 4
Đắk Lắk                  4
Quảng Bình               4
Kon Tum                  3
Hải Dương                3
Yên Bái                  3
N

In [18]:
count_value = df['district'].value_counts()
print(count_value)

Quận 7                  13001
Quận Tân Phú             3413
Quận 2 (TP. Thủ Đức)     3016
                         2211
Quận Bình Thạnh          2127
                        ...  
Thị xã Tây Ninh             1
Huyện Bến Lức               1
Huyện Hàm Thuận Bắc         1
TP. Rạch Giá                1
Huyện Vĩnh Cửu              1
Name: district, Length: 205, dtype: int64


In [19]:
df_test = df

In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64508 entries, 0 to 64561
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          64506 non-null  object 
 1   price          64508 non-null  float64
 2   address        64508 non-null  object 
 3   area           62322 non-null  object 
 4   date           59973 non-null  object 
 5   description    60830 non-null  object 
 6   link_image     64508 non-null  object 
 7   url_page       64508 non-null  object 
 8   direct         4211 non-null   object 
 9   floor          6918 non-null   float64
 10  name_contact   29488 non-null  object 
 11  phone_contact  64506 non-null  object 
 12  city           64508 non-null  object 
 13  district       64508 non-null  object 
dtypes: float64(2), object(12)
memory usage: 7.4+ MB


In [21]:
indexer = recordlinkage.Index()
indexer.block(['area', 'price', 'district', 'city'])
candidates = indexer.index(df_test)

In [22]:
len(candidates)

82395

In [23]:
compare_cl = recordlinkage.Compare()
compare_cl.string("title", "title", method="levenshtein", threshold=0.8, label="title")
compare_cl.string("description", "description", method="levenshtein", threshold=0.7, label="title")

features = compare_cl.compute(candidates, df_test)

In [24]:
matches = features[features.sum(axis=1) == 2]

In [25]:
for (x,y) in matches.index:
    print(df_test['title'][x])
    print(df_test['title'][y])
    print()

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ

CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ
CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ

CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ
CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ

CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ
CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ

CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ
CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ

CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ
CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ

CẦN BÁN GẤP CĂN HỘ MẶT BIỂN TRUNG TÂM QUY NHƠN - FULL NỘI THẤT - GIÁ 1.1 TỶ
CẦN BÁN GẤP CĂN HỘ MẶT BI

In [26]:
def find(parent, i):
    if parent[i] == i:
        return i
    return find(parent, parent[i])

def union(parent, rank, x, y):
    x_root = find(parent, x)
    y_root = find(parent, y)

    if rank[x_root] < rank[y_root]:
        parent[x_root] = y_root
    elif rank[x_root] > rank[y_root]:
        parent[y_root] = x_root
    else:
        parent[y_root] = x_root
        rank[x_root] += 1

def group_related_pairs(pairs):
    parent = {}
    rank = {}

    for pair in pairs:
        x, y = pair

        if x not in parent:
            parent[x] = x
            rank[x] = 0
        if y not in parent:
            parent[y] = y
            rank[y] = 0

        union(parent, rank, x, y)

    groups = {}
    for key in parent.keys():
        root = find(parent, key)
        if root in groups:
            groups[root].append(key)
        else:
            groups[root] = [key]

    return list(groups.values())

In [27]:
list_dup = []

for (x,y) in matches.index:
    list_dup.append([x,y])

groups = group_related_pairs(list_dup)
print(groups)

[[18689, 5298, 63727], [6899, 6647], [103, 19], [5543, 5534], [5302, 5296, 17600], [24552, 6944], [24553, 7136], [64307, 5305], [18994, 5293], [8641, 8640, 8937, 30567], [5188, 5161], [19608, 348], [17334, 5289, 63964], [3723, 3652], [12146, 8965, 13061], [25610, 4485], [43164, 2015], [46048, 40100], [11292, 11256], [8548, 8547, 8565], [7555, 1009], [8509, 8464], [25998, 6279], [46049, 40802, 46534], [13177, 12768, 12773, 13196], [5511, 5499], [9622, 9277, 10285], [5113, 5107], [40054, 37299], [44364, 40458], [46829, 39214], [62067, 62015], [13838, 13822, 13845, 13851, 13856, 13880, 14001], [5540, 5539], [28535, 5929], [17316, 5300, 63794], [8314, 5637], [36392, 36130], [34418, 34352, 34431, 38466], [4448, 4430, 4546], [4484, 4435], [9537, 8076], [2367, 2366], [34421, 34354, 35500, 38525], [26045, 24255], [56423, 34477], [8964, 8797], [3076, 2827], [55792, 29636], [60321, 60071], [63603, 62380], [51103, 50922, 51121], [5553, 5535], [9855, 5670, 9929, 10284], [5743, 3598], [3819, 3604, 

In [28]:
print(len(groups))

1885


In [29]:
list_del = []

for x in groups:
    list_del += x[1:]

print(list_del)

[5298, 63727, 6647, 19, 5534, 5296, 17600, 6944, 7136, 5305, 5293, 8640, 8937, 30567, 5161, 348, 5289, 63964, 3652, 8965, 13061, 4485, 2015, 40100, 11256, 8547, 8565, 1009, 8464, 6279, 40802, 46534, 12768, 12773, 13196, 5499, 9277, 10285, 5107, 37299, 40458, 39214, 62015, 13822, 13845, 13851, 13856, 13880, 14001, 5539, 5929, 5300, 63794, 5637, 36130, 34352, 34431, 38466, 4430, 4546, 4435, 8076, 2366, 34354, 35500, 38525, 24255, 34477, 8797, 2827, 29636, 60071, 62380, 50922, 51121, 5535, 5670, 9929, 10284, 3598, 3604, 3869, 3891, 3608, 3614, 3628, 3694, 3644, 3617, 3697, 7912, 7938, 4014, 3626, 3638, 3668, 4369, 12123, 3727, 45437, 13309, 11031, 3805, 3843, 3848, 3857, 3883, 3929, 7545, 12566, 3927, 3948, 3955, 9294, 11275, 11382, 13733, 35373, 3971, 3979, 3998, 11374, 4013, 11005, 4044, 4081, 4063, 4265, 13462, 13552, 11047, 29646, 4106, 6149, 4115, 5569, 4135, 12740, 4140, 4371, 4439, 4622, 4143, 4432, 4633, 4141, 4153, 4441, 4159, 4145, 10708, 10685, 4163, 4147, 4154, 4434, 4155, 462

In [30]:
print(len(list_del))

2643


In [31]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64508 entries, 0 to 64561
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          64506 non-null  object 
 1   price          64508 non-null  float64
 2   address        64508 non-null  object 
 3   area           62322 non-null  object 
 4   date           59973 non-null  object 
 5   description    60830 non-null  object 
 6   link_image     64508 non-null  object 
 7   url_page       64508 non-null  object 
 8   direct         4211 non-null   object 
 9   floor          6918 non-null   float64
 10  name_contact   29488 non-null  object 
 11  phone_contact  64506 non-null  object 
 12  city           64508 non-null  object 
 13  district       64508 non-null  object 
dtypes: float64(2), object(12)
memory usage: 9.4+ MB


In [32]:
df_test = df_test.drop(index=list_del)

In [33]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61865 entries, 0 to 64561
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          61863 non-null  object 
 1   price          61865 non-null  float64
 2   address        61865 non-null  object 
 3   area           59679 non-null  object 
 4   date           57342 non-null  object 
 5   description    58187 non-null  object 
 6   link_image     61865 non-null  object 
 7   url_page       61865 non-null  object 
 8   direct         3845 non-null   object 
 9   floor          5942 non-null   float64
 10  name_contact   27600 non-null  object 
 11  phone_contact  61863 non-null  object 
 12  city           61865 non-null  object 
 13  district       61865 non-null  object 
dtypes: float64(2), object(12)
memory usage: 7.1+ MB


In [34]:
df_test.head()

,title,price,address,area,date,description,link_image,url_page,direct,floor,name_contact,phone_contact,city,district
0,Căn Hộ 2 PN 55m2 Phan Huy Ích – sát KCN Tân B...,880.0,"Phường 15, Quận Tân Bình, TP Hồ Chí Minh",55.0m2,NaN,NaN,['https://img.homedy.com/store/images/2023/06/...,https://homedy.com/ban-can-ho-quan-tan-binh-tp...,NaN,NaN,Quý,0901.321.245,Hồ Chí Minh,Quận Tân Bình
1,Căn Hộ Q Tân Bình 860tr/sổ riêng-Trọn đời. Tặ...,860.0,"Phường 13, Quận Tân Bình, TP Hồ Chí Minh",60.0m2,NaN,NaN,['https://img.homedy.com/store/images/2023/06/...,https://homedy.com/ban-can-ho-quan-tan-binh-tp...,NaN,NaN,Trần Mai Phương,0931.601.365,Hồ Chí Minh,Quận Tân Bình
2,Căn Hộ 1PN Full Nội Thất 38m2 CẠNH SÔNG SÀI G...,11.5,"Phường Bình An, Quận 2, TP Hồ Chí Minh",38.0m2,NaN,NaN,['https://img.homedy.com/store/images/2023/06/...,https://homedy.com/ban-can-ho-quan-2-tp-ho-chi...,NaN,NaN,Nguyễn Thị Tuyết Hương,0906.022.702,Hồ Chí Minh,Quận 2
3,Bán căn hộ Vũng Tàu Center Thành phố Vũng Tàu...,2650.0,"Phường Thắng Nhì, Thành phố Vũng Tàu, Bà Rịa ...",76.0m2,NaN,- Ngân Hàng tài trợ 70%,['https://img.homedy.com/store/images/2023/03/...,https://homedy.com/ban-can-ho-thanh-pho-vung-t...,NaN,NaN,Son,0937.002.378,Bà Rịa - Vũng Tàu,Thành phố Vũng Tàu
4,Bán căn hộ Thành phố Đà Lạt - Lâm Đồng giá 2....,2900.0,"Phường 10, Thành phố Đà Lạt, Lâm Đồng",56.0m2,NaN,NaN,['https://img.homedy.com/store/images/2023/06/...,https://homedy.com/ban-can-ho-thanh-pho-da-lat...,NaN,NaN,Liêm ngô,0937.500.334,Lâm Đồng,Thành phố Đà Lạt


In [35]:
df_test.to_csv('/content/drive/MyDrive/2022_2/Data_Integration/Train_1_10/data_ok.csv', index=False)